IF THE MODEL ZIP FILE AND LABEL ENCODER FILES ARE UPLOADED

In [3]:
from google.colab import files
uploaded = files.upload()

In [4]:
import shutil

zip_path = "/content/Best_Bert_Model.zip"
extract_path = "/content/Best_Bert_Model"

shutil.unpack_archive(zip_path, extract_path)

print(f"Files extracted to: {extract_path}")

FROM DRIVE

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
import zipfile

zip_path = '/content/drive/MyDrive/Best_Bert_Model.zip'
extract_path = '/content/Best_Bert_Model'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.testzip()  # Check for corruption
    zip_ref.extractall(extract_path)

print(f"Files extracted to: {extract_path}")

Files extracted to: /content/Best_Bert_Model


In [5]:
pkl_drive_file_path = '/content/drive/MyDrive/label_encoder.pkl' #Change this to the right file path if needed

In [ ]:
# Install packages (if needed)
!pip install transformers scikit-learn pandas

In [7]:
import os
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import pickle

model_path = "/content/Best_Bert_Model"  # Path to the unzipped model

tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

# Load label encoder
label_encoder_path = os.path.join(model_path, pkl_drive_file_path)
with open(label_encoder_path, "rb") as f:
    le = pickle.load(f)

# ⚙️ Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [8]:
# 🔮 Prediction function
def predict_intent(query: str):
    inputs = tokenizer(query, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
        intent = le.inverse_transform([predicted_class])[0]

    return intent

In [9]:
query = "I want to reschedule my flight"
predicted_intent = predict_intent(query)
print(f"🔍 Predicted Intent: {predicted_intent}")

🔍 Predicted Intent: Booking, Modifications And Cancellations


In [10]:
predict_intent("What is the capital of India?")

'Irrelevant'

In [11]:
predict_intent("Can I use my HDFC debit card to pay for my SkyWings flight booking?")

'Fares And Payments'

In [12]:
predict_intent("I don't know how to book a SkyWings flight. Can you help me book a flight from Bangalore to Mumbai?")

'Booking, Modifications And Cancellations'

In [13]:
predict_intent("What are the cancellation charges if I cancel my flight scheduled for tomorrow from Delhi to chennai?")

'Booking, Modifications And Cancellations'

In [14]:
predict_intent("Can you explain the process of web-checkin?")

'Check-In And Boarding'

In [15]:
predict_intent("What language will the in-flight announcements be in?")

'Other'

In [16]:
predict_intent("I've lost my luggage. Can you help me find it?")

'Baggage'

In [19]:
predict_intent("I'm vegetariam. What are my meal options on the flight?")

'Passenger Services'

In [26]:
predict_intent("what's the weather like today?")

'Irrelevant'

In [27]:
predict_intent("What're the available flights tomorrow from Bangalore to Delhi?")

'Flight Operations'